# funcX Tutorial

funcX is a Function-as-a-Service (FaaS) platform for science that enables you to convert almost any computing resource into a high-performance function serving device. Deploying a funcX endpoint agent will integrate your resource into the function serving fabric, allowing you to dynamically send, monitor, and receive results from function invocations. funcX is built on top of [Parsl](https://parsl-project.org), allowing you to connect your endpoint to large compute resources via traditional batch queues, where funcX will dynamically provision, use, and release resources on-demand to fulfill function requests.

Here we provide an example of using funcX to register a function and run it on a publicly available tutorial endpoint.

## funcX Client

We start by creating a funcX client to interact with the service. The client is a programmatic means of communicating with the funcX service and allows you to:
- Register functions
- Register containers and execution environments
- Launch registered function against endpoints
- Check the status of launched functions and
- Retrieve outputs from functions

#### Authentication

Creating a client will force an authentication flow where you will be asked to authenticate with Globus Auth. Every interaction with funcX is authenticated to allow us to enforce access control on both functions and endpoints. As part of the authentication process we request access to your identity information (to retrieve your email address) and Globus Groups management access. We require Groups access in order to facilitate sharing.

In [1]:
from funcx.sdk.client import FuncXClient

fxc = FuncXClient()

Here we define a Python function to register with funcX. This function simply sums the input passed to the function.

When defining a function you can specify \*args and \*\*kwargs as inputs. 

##### Note: any dependencies for the function must be specified inside the function body.

In [2]:
def funcx_sum(items):
    return sum(items)

## Registering a function

To use a function with funcX you must first register it with the service. Using `register_function`, you can specify a function to register. You can also optionally include a description of the function.

The registration process will serialize the function body and transmit it to the funcX Web Service.

Registering a function results in a UUID that can be used to invoke the function.

In [3]:
func_uuid = fxc.register_function(funcx_sum,
                                  description="A sleep function")
print(func_uuid)

9975a590-e3a9-4277-941a-e8bd6482be90


## Running a function

To invoke a function you must specify the `function_id`, returned from the registration process, and an `endpoint_id`, to perform the function. Note: here we use the funcX public tutorial endpoint.

The client's `run` function will serialize any \*args and \*\*kwargs, and pass them to the function when invoking it. Therefore, as our example function simply takes an arg input (items) we can specify an input arg and it will be used by the function. Here we define a small list of integers for our function to sum.

The Web service will return the UUID for the executing task. This UUID can be used to check the status of the request and retrieve the result.

In [4]:
endpoint_uuid = '4b116d3c-1703-4f8f-9f6f-39921e5864df' # Public tutorial endpoint

payload = [1, 2, 3, 4, 5]

res = fxc.run(payload, endpoint_id=endpoint_uuid, function_id=func_uuid)
print(res)

283020f3-2761-4816-8357-729471cda00e


You can now retrieve the result of the invocation using `get_result()`. 

##### Note: We remove the task from our database once the result has been retrieved, thus you can only retireve the result once.

In [5]:
fxc.get_result(res)

15